In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split # module to split our data into train and test sets
from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm
import statsmodels.tools 

%matplotlib inline

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# read data and create the dataframe
diamonds = pd.read_csv("https://resources.digitalfutures.com/data-science/diamonds.csv")
#show the head 5 of the dataframe
diamonds.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [ ]:
# show the column cut's unique value
diamonds.cut.unique()

array(['Ideal', 'Premium', 'Good', 'Very Good', 'Fair'], dtype=object)

In [ ]:
# scale the data

to_scale = ['carat','depth','table','price','x','y','z']

st_scaler = StandardScaler()
diamonds_scaled = st_scaler.fit_transform(diamonds[to_scale])
diamonds_scaled = pd.DataFrame(diamonds_scaled, columns = to_scale)

In [ ]:
#encode the data
to_ohe = ['cut','color','clarity']

one_hot_encoded = pd.get_dummies(diamonds[to_ohe],columns = to_ohe, prefix = to_ohe, drop_first = True)

In [ ]:
# join to dataframe, (original cut, color and clarity have been dropped when scale it)
diamonds = diamonds_scaled.join(one_hot_encoded)

In [ ]:
# show the head of the scaled dataframe
diamonds.head()

,carat,depth,table,price,x,y,z,cut_Good,cut_Ideal,cut_Premium,...,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,-1.198168,-0.174092,-1.099672,-0.904095,-1.587837,-1.536196,-1.571129,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1,-1.240361,-1.360738,1.585529,-0.904095,-1.641325,-1.658774,-1.741175,0,0,1,...,0,0,0,0,1,0,0,0,0,0
2,-1.198168,-3.385019,3.375663,-0.903844,-1.498691,-1.457395,-1.741175,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,-1.071587,0.454133,0.242928,-0.902090,-1.364971,-1.317305,-1.287720,0,0,1,...,0,1,0,0,0,0,0,1,0,0
4,-1.029394,1.082358,0.242928,-0.901839,-1.240167,-1.212238,-1.117674,1,0,0,...,0,0,1,0,0,1,0,0,0,0


## Train & Test Spliting

In [ ]:
#show the column names of the dataframe
diamonds.columns

Index(['carat', 'depth', 'table', 'price', 'x', 'y', 'z', 'cut_Good',
       'cut_Ideal', 'cut_Premium', 'cut_Very Good', 'color_E', 'color_F',
       'color_G', 'color_H', 'color_I', 'color_J', 'clarity_IF', 'clarity_SI1',
       'clarity_SI2', 'clarity_VS1', 'clarity_VS2', 'clarity_VVS1',
       'clarity_VVS2'],
      dtype='object')

In [ ]:
# set the columns (except price column) as feature columns
feature_cols = list(diamonds.columns)
feature_cols.remove ('price')
feature_cols

['carat',
 'depth',
 'table',
 'x',
 'y',
 'z',
 'cut_Good',
 'cut_Ideal',
 'cut_Premium',
 'cut_Very Good',
 'color_E',
 'color_F',
 'color_G',
 'color_H',
 'color_I',
 'color_J',
 'clarity_IF',
 'clarity_SI1',
 'clarity_SI2',
 'clarity_VS1',
 'clarity_VS2',
 'clarity_VVS1',
 'clarity_VVS2']

In [ ]:
# split the data
X = diamonds[feature_cols]
y = diamonds['price']

In [ ]:
# train the splited data
from pandas.core.common import random_state

X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, random_state = 20)


## Model Building

In [ ]:
# OLS linear regression summary of the train price to other feature columns
linreg = sm.OLS(y_train, X_train[feature_cols])
results = linreg.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.910
Model:                            OLS   Adj. R-squared (uncentered):              0.910
Method:                 Least Squares   F-statistic:                          1.896e+04
Date:                Thu, 30 Jun 2022   Prob (F-statistic):                        0.00
Time:                        09:49:58   Log-Likelihood:                         -9411.1
No. Observations:               43152   AIC:                                  1.887e+04
Df Residuals:                   43129   BIC:                                  1.907e+04
Df Model:                          23                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
carat             1.3119      0.007    194.661      0.000       1.299       1.325
depth            -0.0495      0.002    -26.182      0.000      -0.053      -0.046
table            -0.0405      0.002    -21.099      0.000      -0.044      -0.037
x                -0.2700      0.010    -25.904      0.000      -0.290      -0.250
y                 0.0104      0.006      1.757      0.079      -0.001       0.022
z                -0.0097      0.007     -1.446      0.148      -0.023       0.003
cut_Good         -0.1405      0.009    -15.409      0.000      -0.158      -0.123
cut_Ideal        -0.1176      0.009    -13.368      0.000      -0.135      -0.100
cut_Premium      -0.1141      0.009    -13.356      0.000      -0.131      -0.097
cut_Very Good    -0.1175      0.009    -13.680      0.000      -0.134      -0.101
color_E          -0.1327      0.005    -25.642      0.000      -0.143      -0.123
color_F          -0.1508      0.005    -28.800      0.000      -0.161      -0.141
color_G          -0.2010      0.005    -39.141      0.000      -0.211      -0.191
color_H          -0.3287      0.005    -60.059      0.000      -0.339      -0.318
color_I          -0.4406      0.006    -70.801      0.000      -0.453      -0.428
color_J          -0.6649      0.008    -86.661      0.000      -0.680      -0.650
clarity_IF        0.6847      0.012     57.903      0.000       0.662       0.708
clarity_SI1       0.2544      0.009     28.919      0.000       0.237       0.272
clarity_SI2       0.0136      0.009      1.537      0.124      -0.004       0.031
clarity_VS1       0.4882      0.009     52.833      0.000       0.470       0.506
clarity_VS2       0.4090      0.009     45.945      0.000       0.392       0.426
clarity_VVS1      0.5978      0.010     57.839      0.000       0.577       0.618
clarity_VVS2      0.5817      0.010     59.719      0.000       0.563       0.601
==============================================================================
Omnibus:                    12505.666   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           970137.654
Skew:                          -0.460   Prob(JB):                         0.00
Kurtosis:                      26.210   Cond. No.                         37.6
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# predict the price with the train data
X_train['y_pred'] = results.predict(X_train[feature_cols])


In [ ]:
# show RMSE 
rmse = statsmodels.tools.eval_measures.rmse(y_train, X_train['y_pred'])

print(rmse)

In [ ]:
## evaluate in Test

linreg_test = sm.OLS(y_test, X_test[feature_cols])
results_t = linreg_test.fit()
results_t.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.911
Model:                            OLS   Adj. R-squared (uncentered):              0.910
Method:                 Least Squares   F-statistic:                              4766.
Date:                Thu, 30 Jun 2022   Prob (F-statistic):                        0.00
Time:                        09:50:29   Log-Likelihood:                         -2149.8
No. Observations:               10788   AIC:                                      4346.
Df Residuals:                   10765   BIC:                                      4513.
Df Model:                          23                                                  
Covariance Type:            nonrobust                                                  
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
carat             1.3907      0.015     94.797      0.000       1.362       1.420
depth            -0.0543      0.004    -14.010      0.000      -0.062      -0.047
table            -0.0237      0.004     -6.407      0.000      -0.031      -0.016
x                -0.8743      0.055    -15.757      0.000      -0.983      -0.766
y                 0.5164      0.056      9.144      0.000       0.406       0.627
z                 0.0101      0.018      0.574      0.566      -0.024       0.045
cut_Good         -0.1909      0.018    -10.899      0.000      -0.225      -0.157
cut_Ideal        -0.1357      0.017     -8.045      0.000      -0.169      -0.103
cut_Premium      -0.1247      0.016     -7.632      0.000      -0.157      -0.093
cut_Very Good    -0.1570      0.017     -9.425      0.000      -0.190      -0.124
color_E          -0.1257      0.010    -12.051      0.000      -0.146      -0.105
color_F          -0.1580      0.010    -15.163      0.000      -0.178      -0.138
color_G          -0.1993      0.010    -19.520      0.000      -0.219      -0.179
color_H          -0.3184      0.011    -29.303      0.000      -0.340      -0.297
color_I          -0.4530      0.012    -37.519      0.000      -0.477      -0.429
color_J          -0.6897      0.015    -44.784      0.000      -0.720      -0.660
clarity_IF        0.6751      0.023     29.181      0.000       0.630       0.720
clarity_SI1       0.2827      0.017     16.646      0.000       0.249       0.316
clarity_SI2       0.0378      0.017      2.210      0.027       0.004       0.071
clarity_VS1       0.5057      0.018     28.390      0.000       0.471       0.541
clarity_VS2       0.4226      0.017     24.498      0.000       0.389       0.456
clarity_VVS1      0.6096      0.020     30.429      0.000       0.570       0.649
clarity_VVS2      0.5808      0.019     30.655      0.000       0.544       0.618
==============================================================================
Omnibus:                     2554.340   Durbin-Watson:                   1.995
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            94300.023
Skew:                           0.404   Prob(JB):                         0.00
Kurtosis:                      17.462   Cond. No.                         54.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
## evaluate in Test

X_test['y_pred'] = results_t.predict(X_test[feature_cols])

rmse = statsmodels.tools.eval_measures.rmse(y_test, X_test['y_pred'])

print(rmse)

0.29532891136448575
